# Imports

In [9]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import os
import warnings
from math import radians, cos, sin, acos

# Carrega os dataframes

In [10]:
# Inputs
cdd = "no"
operacao = "lages"
nome_df_alocacao = f"df_alocacao_{operacao}.csv"
nome_df_clientes = f"clientes_{operacao}.csv"

# Ranges de on 

In [11]:
# Caminho
PATH = os.path.join(os.getcwd(), "Inputs", nome_df_alocacao)

# Carrega o dataframe
df_alocacao = pd.read_csv(PATH, sep=",")
df_alocacao.head()

,cod_unb,cod_pdv,cod_setor,freq_roteirizador,dia_visita,status_planner
0,624004,736,301,S,Ter,cliente_roteirizado
1,624004,9317,402,S,Qua,cliente_roteirizado
2,624004,6672,402,Q,Qui,cliente_roteirizado
3,624004,10881,402,S,Ter,cliente_roteirizado
4,624004,3343,403,S,Ter,cliente_roteirizado


In [12]:
df_alocacao["cod_setor"].value_counts()

cod_setor
404    210
405    190
406    187
402    183
403    181
407    174
401    173
301     80
303     80
302     78
304     34
Name: count, dtype: int64

In [13]:
# Count por setor
counts = df_alocacao["cod_setor"].value_counts()

# Cria uma nova coluna "perfil" baseada no count
df_alocacao["perfil"] = df_alocacao["cod_setor"].apply(lambda x: "on_trade" if counts[x] > 140 else "outro")

In [14]:
# Cria a coluna cod_router se for CDD
cod_unb = df_alocacao["cod_unb"][0]

if cdd == "yes":
    df_alocacao["cod_router"] = df_alocacao["cod_pdv"].map(lambda x: str(cod_unb) + str(x).zfill(5))
    df_alocacao["cod_router"] = df_alocacao["cod_router"].astype(int)

else:
    df_alocacao.rename(columns={"cod_pdv": "cod_router"}, inplace=True)

# Printa o df
df_alocacao.head()

,cod_unb,cod_router,cod_setor,freq_roteirizador,dia_visita,status_planner,perfil
0,624004,736,301,S,Ter,cliente_roteirizado,outro
1,624004,9317,402,S,Qua,cliente_roteirizado,on_trade
2,624004,6672,402,Q,Qui,cliente_roteirizado,on_trade
3,624004,10881,402,S,Ter,cliente_roteirizado,on_trade
4,624004,3343,403,S,Ter,cliente_roteirizado,on_trade


In [15]:
# Mapa dtype
dict_dtype = {
    "cod_router": int
}

In [16]:
# Caminho
PATH = os.path.join(os.getcwd(), "Inputs", nome_df_clientes)

# Carrega o dataframe
df_clientes = pd.read_csv(PATH, sep=";", dtype=dict_dtype)

# Seleciona latitude e longitude válidas, bem como código cliente
if cdd == "yes":
    df_clientes = df_clientes[df_clientes["Código Cliente"].str.startswith(str(cod_unb))]
    
df_clientes = df_clientes.loc[(df_clientes["Latitude"] != "-") & (df_clientes["Longitude"] != "-")]

df_clientes.head()

,Código Cliente,Nome Cliente,Tipo Cliente,Prioridade,Tempo Espera,Setor,Endereço,Cidade,Estado,Latitude,...,Data última compra,Data penúltima compra,Macro região,Setor Planner,Veículos exclusivos,Tipos Veículos exclusivos,Prioridade Tipo Cliente,Origem Tempo Espera,Grupos de rota exclusiva,Unnamed: 28
0,5000,** MERCEARIA AVENIDA,-,Normal,00:05:00,LAGES 5,DOUTOR AUJOR LUZ 679,LAGES,SC,"-27,838870",...,19/10/2024,05/10/2024,-,Lages,-,-,-,Tempo do setor,-,NaN
1,1208,***AABB,-,Normal,00:10:59,LAGES CENTRO,"Avenida Papa João XXIII, 2298",Lages,SC,"-27,808182",...,-,-,-,Lages,-,-,-,Cadastro do cliente,-,NaN
2,1392,***ARENA FUTEBOL CLU,-,Normal,00:10:23,Sem setor cadastrado,OTACILIO VIEIRA DA COSTA 363,LAGES,SC,"-28,033198",...,-,-,-,Lages,-,-,-,Cadastro do cliente,-,NaN
3,2376,***BAITO LANCHES,-,Normal,00:05:00,CORREIA PINTO 2,TANCREDO NEVES 737,CORREIA PINTO,SC,"-27,588687",...,02/10/2024,25/09/2024,-,Lages,-,-,-,Tempo do setor,-,NaN
4,3143,***BAR FIGUEIRINHA,-,Normal,00:07:46,LAGES 1,SEBASTIAO CAMARGO 85,LAGES,SC,"-27,793500",...,01/11/2024,25/10/2024,-,Lages,-,-,-,Cadastro do cliente,-,NaN


# Data wrangling

In [17]:
# Ajustes na base
df_clientes = df_clientes.rename(columns={"Código Cliente": "cod_router", "Latitude": "latitude", "Longitude": "longitude"})
df_clientes = df_clientes.dropna(subset=["cod_router"])
df_clientes["latitude"] = df_clientes["latitude"].str.replace(",", ".")
df_clientes["longitude"] = df_clientes["longitude"].str.replace(",", ".")
df_clientes["latitude"] = df_clientes["latitude"].astype(float)
df_clientes["longitude"] = df_clientes["longitude"].astype(float)
df_clientes["cod_router"] = df_clientes["cod_router"].astype(int)

df_clientes[["cod_router", "latitude", "longitude"]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 5818 entries, 0 to 5929
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   cod_router  5818 non-null   int32  
 1   latitude    5818 non-null   float64
 2   longitude   5818 non-null   float64
dtypes: float64(2), int32(1)
memory usage: 159.1 KB


In [18]:
# Realizar o merge mantendo apenas as colunas "latitude" e "longitude" de df_clientes
df_planner = pd.merge(df_alocacao, df_clientes[["cod_router", "latitude", "longitude"]], on="cod_router", how="left")
df_planner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1570 entries, 0 to 1569
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cod_unb            1570 non-null   int64  
 1   cod_router         1570 non-null   int64  
 2   cod_setor          1570 non-null   int64  
 3   freq_roteirizador  1570 non-null   object 
 4   dia_visita         1570 non-null   object 
 5   status_planner     1570 non-null   object 
 6   perfil             1570 non-null   object 
 7   latitude           1570 non-null   float64
 8   longitude          1570 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 110.5+ KB


In [19]:
# Faz o slice apenas nos clientes roteirizados
df_roterizados = df_planner.loc[df_planner["status_planner"] == "cliente_roteirizado"]

# Faz o filtro para considerar apenas setores on_trade
df_ontrade = df_roterizados.loc[df_roterizados["perfil"] == "on_trade"]

In [20]:
# Função para calcular a mediana
def percentile_approx(column, percentile):
    return np.percentile(column, percentile)

# Agrupar e calcular a mediana aproximada de cada setor e dia de visita
df_cms = df_ontrade.groupby(["cod_setor", "dia_visita"])\
                       .agg(lat_cm=("latitude", lambda x: percentile_approx(x, 50)),
                            long_cm=("longitude", lambda x: percentile_approx(x, 50)))\
                       .reset_index()

df_cms.head()

,cod_setor,dia_visita,lat_cm,long_cm
0,401,Qua,-27.814459,-50.323011
1,401,Qui,-27.816289,-50.331848
2,401,Seg,-27.810441,-50.286112
3,401,Sex,-27.826466,-50.334809
4,401,Ter,-27.799407,-50.295572


In [21]:
# Cria um dataframe com os novos cadastros
df_ncs = df_planner.loc[df_planner["status_planner"] == "novo_cliente"]

# Seleciona as colunas importantes e retira quem tem coordenada nula
cod_pdv_values = df_ncs[["cod_router", "latitude", "longitude"]]
cod_pdv_values = cod_pdv_values.loc[cod_pdv_values["latitude"].isna() == False]

In [22]:
# Função para calcular a distância
def calculate_distance(lat_a, long_a, lat_b, long_b):
    dist_m = acos(cos(radians(90 - lat_a)) * cos(radians(90 - lat_b)) 
                  + sin(radians(90 - lat_a)) * sin(radians(90 - lat_b)) 
                  * cos(radians(long_a - long_b))) * 6371 * 1000
    return dist_m

def calculate_distance_row(row):
    return calculate_distance(row["lat_cm"], row["long_cm"],
                              row["latitude"], row["longitude"])

In [23]:
df_distancias = cod_pdv_values.assign(key=1).merge(df_cms.assign(key=1), on="key").drop("key", axis=1)

# Aplicar a função de distância para calcular a distância entre cada PDV e cada centro de massa
df_distancias["dist_cm_dia"] = df_distancias.apply(calculate_distance_row, axis=1)
df_distancias.head()

,cod_router,latitude,longitude,cod_setor,dia_visita,lat_cm,long_cm,dist_cm_dia
0,6709,-27.779416,-50.307756,401,Qua,-27.814459,-50.323011,4175.540163
1,6709,-27.779416,-50.307756,401,Qui,-27.816289,-50.331848,4735.710189
2,6709,-27.779416,-50.307756,401,Seg,-27.810441,-50.286112,4053.888847
3,6709,-27.779416,-50.307756,401,Sex,-27.826466,-50.334809,5869.538507
4,6709,-27.779416,-50.307756,401,Ter,-27.799407,-50.295572,2525.428161


In [24]:
# Encontrar a distância mínima por "cod_router" e "cod_setor"
df_minimo = df_distancias.sort_values(by=["dist_cm_dia"]).groupby(["cod_router"]).agg(melhor_setor=("cod_setor", "first"), melhor_dia=("dia_visita", "first")).reset_index()
df_minimo

,cod_router,melhor_setor,melhor_dia
0,765,405,Seg
1,1636,404,Qua
2,1901,402,Ter
3,2965,405,Ter
4,4271,402,Ter
...,...,...,...
76,11754,402,Qua
77,11755,404,Sex
78,12603,407,Ter
79,15122,406,Seg


In [25]:
sorted(df_minimo["melhor_setor"].unique())

[401, 402, 403, 404, 405, 406, 407]

In [26]:
# Faz o merge do df_minimo com o df_ncs
df_ncs_novo = df_minimo.merge(df_ncs.drop(["cod_setor", "dia_visita", "latitude", "longitude"], axis=1), on="cod_router", how="inner")\
    .rename(columns={"melhor_setor": "cod_setor", "melhor_dia": "dia_visita"})

# Reorganiza as colunas
df_ncs_novo = df_ncs_novo[["cod_unb", "cod_router", "cod_setor", "freq_roteirizador", "dia_visita", "status_planner"]]

# Dropa as colunas que não precisamos e reorganiza as colunas
df_roterizados.drop(["latitude", "longitude", "perfil"], axis=1, inplace=True)
df_roterizados = df_roterizados[["cod_unb", "cod_router", "cod_setor", "freq_roteirizador", "dia_visita", "status_planner"]]

C:\Users\gabri\AppData\Local\Temp\ipykernel_12036\4273482948.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_roterizados.drop(["latitude", "longitude", "perfil"], axis=1, inplace=True)


In [27]:
# Faz a concatenação dos dataframes
df_final = pd.concat([df_roterizados, df_ncs_novo], axis=0)

In [28]:
melhor_alocacao = f"melhor_alocacao_{operacao}"
df_final.to_csv(os.path.join(os.getcwd(), "Outputs", f"{melhor_alocacao}.txt"), header=None, index=None, sep=";", mode="a")